In [ ]:
import time
import tweepy                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  
from tweepy import OAuthHandler                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                
from tweepy.streaming import StreamListener                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    
from tweepy import Stream                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      
import kafka                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   
import json                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    
import re                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      
from kafka import KafkaProducer 

In [ ]:
consumer_key = " "
consumer_secret = " "
access_token = " "
access_token_secret = " "

auth = OAuthHandler(consumer_key, consumer_secret)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             
auth.set_access_token(access_token, access_token_secret)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       
api = tweepy.API(auth) 

In [ ]:
producer = KafkaProducer(bootstrap_servers=["kafka:9092"])
class Listener(tweepy.StreamListener):                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         
    def on_status(self, status):                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         
        if status.coordinates is not None :                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            
            tweet = {'username': status.user.screen_name,
                    'ora': status.created_at.strftime("%Y-%m-%d %H:%M:%S"),
                    'coords': status.coordinates['coordinates'],
                    'testo': status.text,
                    'entities': status.entities,
                    'lingua': status.lang,
                    'user_location': status.user.location}                                                                                                                                                                                                                                                                                                                               
            data_to_kafka = json.dumps(tweet).encode('utf-8') 
            #print('ole')
            producer.send("nazioni", data_to_kafka)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             
        else:                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          
            tweet = {'username': status.user.screen_name,
                    'ora': status.created_at.strftime("%Y-%m-%d %H:%M:%S"),
                    'coords': None,
                    'testo': status.text,
                    'entities': status.entities,
                    'lingua': status.lang,
                    'user_location': status.user.location}                                                                                                                                                                                                                                                                                                                                
            data_to_kafka = json.dumps(tweet).encode('utf-8') 
            #print('ole')
            producer.send("nazioni", data_to_kafka)

In [ ]:
while True:                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   
    try:                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               
        stream = Listener                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              
        stream = Stream(auth = api.auth, listener=Listener())                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            
        stream.filter(track=["#theresnoplanetb","#worldenvironmentday","#fridaysforfuture","#climatechange","#savetheplanet",
                            "#WED","#worldearthday","#worldoceanday","#changementclimatique", "#cambiamentoclimatico", "#cambioclimático",
                            "#気候変動","#Klimawandel", "#plasticfree"])                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            
    except Exception as e:
        print(e)
        print("ERRORE")
        time.sleep(900) 